<a href="https://colab.research.google.com/github/vjumpkung/vjump-sd-webui-colab/blob/main/vjumpkung_A1111_Stable_Diffusion_WebUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# vjumpkung Stable Diffusion WebUI Colab

## แจ้งปัญหาการใช้งาน

https://discord.com/users/250567674504019968

discord username : vjump

### สิ่งที่ต้องการ

- ต้องใช้ Colab Pro หรือ Paid Service เท่านั้น Colab Free ไม่สามารถใช้งานได้

### คู่มือการใช้งาน

- https://vjump-sd-download-list-docs.vercel.app/a1111

### Website แหล่งรวม SD Model List เพื่อสำหรับการใช้งาน Colab นี้

https://vjump-sd-download-list.vjumpkung.dynv6.net/download_list/
### Extension List
- batchlinks-webui
- a1111-sd-webui-tagcomplete
- sd-webui-infinite-image-browsing
- sd-webui-controlnet
- sd-webui-depth-lib
- adetailer
- sd-webui-reactor
- sd-webui-photopea-embed
- sd-civitai-browser-plus
- ultimate-upscale-for-automatic1111

### Credits

- [PromptAlchemist Colab](https://colab.research.google.com/github/poomshift/PromptAlchemist_SD_Webui_Colab/blob/main/PromptAlchemist_SD_Webui_Colab.ipynb) for some pre-install essentials extension.


In [ ]:
#@title # Setup Environment
import os
import sys
from google.colab import drive
from IPython.display import clear_output
import requests
!mkdir -p /content/Downloads
#@markdown ✔️ for save entire folder unless it will save only generated images.
USE_GOOGLE_DRIVE = False  #@param {type:"boolean"}
if 'local_storage' in globals():
    pass
else:
    local_storage = []

SELECT_WEBUI_TYPE = "A1111"

WORKSPACE = "/content/stable-diffusion-webui"
if SELECT_WEBUI_TYPE == "A1111":
    WORKSPACE = "/content/stable-diffusion-webui"

!echo -= Mounting Google Drive =-
drive.mount('/content/drive')

if USE_GOOGLE_DRIVE:
    !echo -= Using Google Drive to Save WebUI =-
    if SELECT_WEBUI_TYPE == "A1111":
        WORKSPACE = "/content/drive/MyDrive/sd-a1111/stable-diffusion-webui"

if SELECT_WEBUI_TYPE == "A1111":
    if not os.path.exists(WORKSPACE):
        !echo -= Initial setup A1111 =- && git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui.git {WORKSPACE} &> /dev/null
%cd $WORKSPACE
if SELECT_WEBUI_TYPE == "A1111":
    basic_config = '''
        "quicksettings_list": [
            "sd_model_checkpoint","sd_vae","CLIP_stop_at_last_layers"
        ]
    '''
    !git checkout tags/v1.10.1 &> /dev/null
    with open(f"{WORKSPACE}/config.json","w") as fp:
        fp.write(basic_config)
# save images to drive
if not USE_GOOGLE_DRIVE:
    savedir = '''
    {
        "outdir_txt2img_samples": "/content/drive/MyDrive/sd-a1111-outputs/txt2img-images",
        "outdir_img2img_samples": "/content/drive/MyDrive/sd-a1111-outputs/img2img-images",
        "outdir_extras_samples": "/content/drive/MyDrive/sd-a1111-outputs/extras-images",
        "outdir_grids": "",
        "outdir_txt2img_grids": "/content/drive/MyDrive/sd-a1111-outputs/txt2img-grids",
        "outdir_img2img_grids": "/content/drive/MyDrive/sd-a1111-outputs/img2img-grids",
        "outdir_save": "/content/drive/MyDrive/sd-a1111-outputs/log/images",
        "outdir_init_images": "/content/drive/MyDrive/sd-a1111-outputs/init-images",
        "quicksettings_list": [
            "sd_model_checkpoint","sd_vae","CLIP_stop_at_last_layers"
        ]
    }
    '''
    with open(f"{WORKSPACE}/config.json","w") as fp:
        fp.write(savedir)
!echo "Install Dependencies"
if not os.path.exists(f"/content/venv"):
    !apt install python3.10-venv &> /dev/null
    !python -m venv /content/venv
    !/content/venv/bin/pip install -q ipykernel
    !/content/venv/bin/pip install -q -U xformers --index-url https://download.pytorch.org/whl/cu121
    !/content/venv/bin/pip install -q -r requirements_versions.txt
    !/content/venv/bin/pip install -q matplotlib==3.8.4 insightface==0.7.3 albumentations==1.4.3 basicsr
    !/content/venv/bin/pip install -q onnx onnxruntime
    !/content/venv/bin/pip install -q onnxruntime-gpu --extra-index-url https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/
#@markdown # Install Extension
enable_controlnet = False #@param{"type":"boolean"}
!mkdir -p {WORKSPACE}/models/ControlNet
!mkdir -p {WORKSPACE}/models/ESRGAN
if not os.path.exists(f"{WORKSPACE}/extensions/batchlinks-webui"):
    !echo "Install Extension : batchlinks-webui"
    !git clone https://github.com/etherealxx/batchlinks-webui.git {WORKSPACE}/extensions/batchlinks-webui &> /dev/null
if not os.path.exists(f"{WORKSPACE}/extensions/a1111-sd-webui-tagcomplete"):
    !echo "Install Extension : a1111-sd-webui-tagcomplete"
    !git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete.git {WORKSPACE}/extensions/a1111-sd-webui-tagcomplete &> /dev/null
if not os.path.exists(f"{WORKSPACE}/extensions/sd-webui-infinite-image-browsing"):
    !echo "Install Extension : sd-webui-infinite-image-browsing"
    !git clone https://github.com/zanllp/sd-webui-infinite-image-browsing.git {WORKSPACE}/extensions/sd-webui-infinite-image-browsing &> /dev/null
if (not os.path.exists(f"{WORKSPACE}/extensions/sd-webui-controlnet")) and (not os.path.exists(f"{WORKSPACE}/extensions/sd-webui-depth-lib")) and SELECT_WEBUI_TYPE == "A1111":
    !echo "Install Extension : ControlNet and Depthlib"
    if enable_controlnet:
        if SELECT_WEBUI_TYPE == "A1111":
            !git clone https://github.com/Mikubill/sd-webui-controlnet.git {WORKSPACE}/extensions/sd-webui-controlnet &> /dev/null
            !git clone https://github.com/jexom/sd-webui-depth-lib.git {WORKSPACE}/extensions/sd-webui-depth-lib &> /dev/null
else:
    print("WebUI Forge Does not need Controlnet Extension")
if not os.path.exists(f"{WORKSPACE}/extensions/adetailer"):
    !echo "Install Extension : adetailer"
    !git clone https://github.com/Bing-su/adetailer.git {WORKSPACE}/extensions/adetailer &> /dev/null
if not os.path.exists(f"{WORKSPACE}/extensions/sd-webui-reactor"):
    !echo "Install Extension : sd-webui-reactor"
    !git clone https://github.com/Gourieff/sd-webui-reactor.git {WORKSPACE}/extensions/sd-webui-reactor &> /dev/null
if not os.path.exists(f"{WORKSPACE}/extensions/sd-webui-photopea-embed"):
    !echo "Install Extension : sd-webui-photopea-embed"
    !git clone https://github.com/yankooliveira/sd-webui-photopea-embed.git {WORKSPACE}/extensions/sd-webui-photopea-embed &> /dev/null
if not os.path.exists(f"{WORKSPACE}/extensions/sd-civitai-browser-plus"):
    !echo "Install Extension : sd-civitai-browser-plus"
    !git clone https://github.com/BlafKing/sd-civitai-browser-plus {WORKSPACE}/extensions/sd-civitai-browser-plus &> /dev/null
if not os.path.exists(f"{WORKSPACE}/extensions/ultimate-upscale-for-automatic1111"):
    !echo "Install Extension : ultimate-upscale-for-automatic1111"
    !git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111.git {WORKSPACE}/extensions/ultimate-upscale-for-automatic1111 &> /dev/null
if not os.path.exists(f"{WORKSPACE}/extensions/model-keyword"):
    !echo "Install Extension : model-keyword"
    !git clone https://github.com/mix1009/model-keyword.git &> /dev/null
if not os.path.exists(f"{WORKSPACE}/repositories/generative-models"):
    !echo "Install Extension : generative-models"
    !git clone https://github.com/Stability-AI/generative-models.git {WORKSPACE}/repositories/generative-models &> /dev/null

#@markdown # Download Controlnet Model
#@markdown สำหรับ SD 1.5

download_controlnet = False #@param {type:"boolean"}

def download_ctrl(Type:str,link:str):
    !mkdir -p /content/Downloads
    URL = link
    if Type == "embeddings":
        !wget -q --show-progress --content-disposition {URL} -P /content/Downloads
        !mv /content/Downloads/* {WORKSPACE}/embeddings/
        return
    if not os.path.exists(f"{WORKSPACE}/models/{Type}/"):
        !mkdir -p {WORKSPACE}/models/{Type}
    !wget -q --show-progress --content-disposition {URL} -P /content/Downloads
    !mv /content/Downloads/* {WORKSPACE}/models/$Type/
controlnet_lst = ['control_v11e_sd15_ip2p.pth',
                  'control_v11e_sd15_ip2p.yaml',
                  'control_v11e_sd15_shuffle.pth',
                  'control_v11e_sd15_shuffle.yaml',
                  'control_v11f1e_sd15_tile.pth',
                  'control_v11f1e_sd15_tile.yaml',
                  'control_v11f1p_sd15_depth.pth',
                  'control_v11f1p_sd15_depth.yaml',
                  'control_v11p_sd15_canny.pth',
                  'control_v11p_sd15_canny.yaml',
                  'control_v11p_sd15_inpaint.pth',
                  'control_v11p_sd15_inpaint.yaml',
                  'control_v11p_sd15_lineart.pth',
                  'control_v11p_sd15_lineart.yaml',
                  'control_v11p_sd15_mlsd.pth',
                  'control_v11p_sd15_mlsd.yaml',
                  'control_v11p_sd15_normalbae.pth',
                  'control_v11p_sd15_normalbae.yaml',
                  'control_v11p_sd15_openpose.pth',
                  'control_v11p_sd15_openpose.yaml',
                  'control_v11p_sd15_scribble.pth',
                  'control_v11p_sd15_scribble.yaml',
                  'control_v11p_sd15_seg.pth',
                  'control_v11p_sd15_seg.yaml',
                  'control_v11p_sd15_softedge.pth',
                  'control_v11p_sd15_softedge.yaml',
                  'control_v11p_sd15s2_lineart_anime.pth',
                  'control_v11p_sd15s2_lineart_anime.yaml']
if download_controlnet:
    for i in controlnet_lst:
        download_ctrl("ControlNet", f"https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/{i}?download=true")
    print("Download Controlnet Model Completed")

def download(Type:str,link:str):
    URL = link
    if not os.path.exists(f"{WORKSPACE}/models/{Type}/"):
        !mkdir -p {WORKSPACE}/models/{Type}
    !wget -q --show-progress --content-disposition {URL} -P /content/Downloads
    exit_code = get_ipython().__dict__['user_ns']['_exit_code']
    if exit_code != 0:
        print(f"Cannot Download {link}")
        return False
    if Type == "embeddings":
        !wget -q --show-progress --content-disposition {URL} -P /content/Downloads
        !mv /content/Downloads/* {WORKSPACE}/embeddings/
        return True
    !mv /content/Downloads/* {WORKSPACE}/models/$Type/
    print(f"Download {Type} from {link} completed")
    return True

# @markdown ### Download Model from Link รวมรายการ Download
# @markdown สามารถค้นหาได้จาก website นี้ https://vjump-sd-download-list.vjumpkung.dynv6.net/download_list/ หรือ Link ที่เก็บไว้จากการสร้าง URL ในขั้นตอน Generate Download List URL
# @markdown <br>คู่มือการใช้งาน SD Download List https://vjump-sd-download-list-docs.vercel.app/sd-dl-list
DOWNLOAD_LIST_URL = "" #@param {type:"string"}
# @markdown ### CivitAI API Keys (สำหรับ Download Model ที่ต้องการ Login)
# @markdown วิธีการขอ CivitAI API Key https://vjump-sd-download-list-docs.vercel.app/others/civit
CIVITAI_API_KEYS = "" #@param {type: "string"}

A1111_map = {
    'checkpoints':'Stable-diffusion',
    'controlnet':'ControlNet',
    'embeddings':'embeddings',
    'hypernetworks':'hypernetworks',
    'vae':'VAE',
    'loras':'Lora',
    'upscale_models':'ESRGAN'
}

if DOWNLOAD_LIST_URL != "":
    r = requests.get(DOWNLOAD_LIST_URL).json()
    if "download_list" not in r.keys() or r["ui_type"] not in ["comfy","A1111"]:
        print("Invalid URL")
    else:
        for i in r["download_list"]:
            if r["ui_type"] == "comfy":
                if i["DownloadModelType"] in A1111_map.keys():
                    i["DownloadModelType"] = A1111_map[i["DownloadModelType"]]
            if CIVITAI_API_KEYS != "" and i["source"] == "CivitAI":
                download(Type=i["DownloadModelType"],link=i["downloadUrl"]+f"?token={CIVITAI_API_KEYS}")
            else:
                download(Type=i["DownloadModelType"],link=i["downloadUrl"])
clear_output()
print(f"Setup Environment and Extension Completed")

In [ ]:
# @title # Download Model (โหลด Model ใหม่)
# @markdown ## รองรับ model จาก civitai, huggingface
#@markdown - [วิธีการ Download Model จากเว็บ CivitAI](https://vjump-sd-download-list-docs.vercel.app/a1111#21-%E0%B8%A7%E0%B8%B4%E0%B8%98%E0%B8%B5-download-%E0%B8%88%E0%B8%B2%E0%B8%81%E0%B9%80%E0%B8%A7%E0%B9%87%E0%B8%9A-civitai)
#@markdown - [วิธีการ Download Model จากเว็บ Huggingface](https://vjump-sd-download-list-docs.vercel.app/a1111#22-%E0%B8%A7%E0%B8%B4%E0%B8%98%E0%B8%B5-download-%E0%B8%88%E0%B8%B2%E0%B8%81%E0%B9%80%E0%B8%A7%E0%B9%87%E0%B8%9A-huggingface)

# @markdown #### ทุกๆ การ download จะมีการเก็บประวัติการ download เอาไว้ใน colab ซึ่งถ้าต้องการเก็บประวัติเอาไว้สามารถกด run Generate Download List URL เพื่อสร้าง link ที่เก็บประวัติการ download ไว้ได้
import requests
from IPython import get_ipython
from urllib.parse import urlparse
from pprint import pp
import os
def download(Type:str,link:str):
    URL = link
    if not os.path.exists(f"{WORKSPACE}/models/{Type}/"):
        !mkdir -p {WORKSPACE}/models/{Type}
    !wget -q --show-progress --content-disposition {URL} -P /content/Downloads
    exit_code = get_ipython().__dict__['user_ns']['_exit_code']
    if exit_code != 0:
        print(f"Cannot Download {link}")
        return False
    !mv /content/Downloads/* {WORKSPACE}/models/$Type/
    clear_output()
    print(f"Download {Type} from {get_down['downloadUrl']} completed")
    return True

def get_url_link(url):
    if "civitai" in url:
        parsedUrl = urlparse(url)
        modelId = int(parsedUrl.path.split("/")[2])
        versionId = None
        if parsedUrl.query != "":
            versionId = int(parsedUrl.query.split("=")[-1])
        getUrl = requests.get(f"https://civitai.com/api/v1/models/{modelId}")
        modelJson = {}
        if getUrl.status_code == 200:
            modelJson = getUrl.json()
        else:
            print("Invalid CivitAI URL")
            return
        getModelInfo = modelJson["modelVersions"][0]
        for i in modelJson["modelVersions"]:
            if i["id"] == versionId:
                getModelInfo = i
                break
        fn = ""
        durl = ""
        sortedImagePreview = getModelInfo["images"]
        previewImage = sortedImagePreview[0]

        for i in getModelInfo["files"]:
            if "primary" in i.keys():
                if i["primary"] == True:
                    fn = i["name"]
                    durl = i["downloadUrl"]
                    break
        res = {
            "model_id": modelJson["id"],
            "source": "CivitAI",
            "type": modelJson["type"],
            "baseModel": getModelInfo["baseModel"],
            "name": modelJson["name"],
            "version": getModelInfo["name"],
            "modellink": url,
            "fileName": fn,
            "downloadUrl": durl,
            "previewImage": previewImage,
        }
        return res
    elif "huggingface" in url:
        parsedUrl = urlparse(url)
        durl = url.replace("blob", "resolve") + "?download=true"
        res = {
            "model_id": None,
            "source": "huggingface",
            "type": None,
            "baseModel": None,
            "name": parsedUrl.path.split("/")[-1],
            "version": None,
            "modellink": url,
            "fileName": parsedUrl.path.split("/")[-1],
            "downloadUrl": durl,
            "previewImage": None,
        }
        return res
    else:
        print("Invalid URL")
Type = "Stable-diffusion" #@param ['Stable-diffusion', 'ControlNet', 'embeddings', 'hypernetworks', 'VAE','Lora','ESRGAN','GFPGAN','reactor','adetailer']
URL = "" #@param {type:"string"}
# @markdown ### CivitAI API Keys (สำหรับ Download Model ที่ต้องการ Login)
CIVITAI_API_KEYS = "" #@param {type: "string"}
#@markdown วิธีการขอ CivitAI API Key https://vjump-sd-download-list-docs.vercel.app/others/civit <br>
#@markdown *โปรดระบุ Type ให้ถูกต้องเพื่อให้ model ที่ download มาอยู่ถูก folder ตามประเภทของ model ที่ได้กำหนดไว้
get_down = get_url_link(URL)
if (get_down == "Invalid URL"):
    raise
get_down["DownloadModelType"] = Type

if CIVITAI_API_KEYS != "" and get_down["source"] == "CivitAI":
    if download(Type, get_down["downloadUrl"]+f"?token={CIVITAI_API_KEYS}"):
        if get_down not in local_storage:
            local_storage.append(get_down)
        print(f"Saved {Type} - {URL} to download list")
else:
    if download(Type, get_down["downloadUrl"]):
        if get_down not in local_storage:
            local_storage.append(get_down)
        print(f"Saved {Type} - {URL} to download list")

In [ ]:
from random import randint
#@title # Generate Download List URL (สำหรับการนำมาใช้ครั้งถัดไป)
#@markdown ### สร้าง url ที่เก็บประวัติการ download model ต่างๆ เพื่อเก็บไว้ใช้ในครั้งถัดไป
#@markdown หัวข้อ (ไม่บังคับ)
Title = "" #@param {type : "string"}
#@markdown คำอธิยาย (ไม่บังคับ)
Description = "" #@param {type : "string"}
#@markdown ชื่อผู้ใช้ (ไม่บังคับ)
Author = "" #@param {type : "string"}
#@markdown ตั้งค่าการมองเห็น (ติ๊กถูก ✅ หมายถึงเห็นทุกคน ถ้าไม่ได้ติ๊กถูกจะเห็นเฉพาะคนที่มีลิ้งเท่านั้น)
isPublic = False #@param {type : "boolean"}
#@markdown กด run เพื่อสร้าง link ที่รวมรายการ download เพื่อเก็บไว้ใช้ในครั้งถัดไป

if Title == "":
    Title = f"Share Download List {hex(randint(10000,1000000000))[2:]}"
if Description == "":
    Description = f"-"
if Author == "":
    Author = "anonymous"

print("----- Recent Download List -----")
for i in local_storage:
    print(f'Model : {i["name"]} Type : {i["DownloadModelType"]}')
print("--------------------------------")

requrl = "https://sd-download-list-api.vjumpkung.dynv6.net/api/create_download_list_link"
if len(local_storage) > 0:
    req = requests.post(requrl,json={
    "title": Title,
    "description": Description,
    "author": Author,
    "ui_type": "A1111",
    "downloaded_list": local_storage,
    "isPublic": isPublic
    })
    print(f"Keep this url for download every model later.")
    print(f"URL : {req.json()['url']}")
else:
    print("Recent Download Model list not found, Please Download some model")

In [ ]:
# @title # Start Stable Diffusion WebUI A1111
%cd {WORKSPACE}
!/content/venv/bin/python launch.py --enable-insecure-extension-access --theme dark --share --xformers